In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
print('tf    version: {}'.format(tf.__version__) )
print('keras version: {}'.format(keras.__version__) )
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

tf    version: 2.0.0
keras version: 2.2.4-tf
matplotlib 3.0.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
input_filepath = './data/shakespeare.txt'

text = open(input_filepath, 'r').read()

In [3]:
vocab = sorted(set(text))
char2idx = {char:idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)


In [4]:
%%time
text_as_int = np.array([char2idx[c] for c in text])


CPU times: user 146 ms, sys: 12.4 ms, total: 159 ms
Wall time: 157 ms


In [5]:
print(len(text_as_int))

print(text_as_int[:10])
print(text[:10])

1115394
[18 47 56 57 58  1 15 47 58 47]
First Citi


In [6]:
%%time
def split_input_target(id_text):
    """
    abcde -> abcd, bcde, 输入和输出
    """
    return id_text[0:-1], id_text[1:]

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100

seq_dataset = char_dataset.batch(seq_length +1 , drop_remainder=True)



# for ch_id in char_dataset.take(2):
#     print(ch_id, idx2char[ch_id.numpy()])
    
# for seq_id in seq_dataset.take(2):
#     print(seq_id)
#     print(repr(''.join(idx2char[seq_id.numpy()])))


CPU times: user 59.2 ms, sys: 114 ms, total: 173 ms
Wall time: 190 ms


In [7]:
seq_dataset = seq_dataset.map(split_input_target)

for item_input, item_output in seq_dataset.take(2):
    print(item_input.numpy())
    print(item_output.numpy())



[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43
 39 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49
  6  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10
  0 37 53 59]
[47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43  1
 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 46 43 56  6  1 46 43 39
 56  1 51 43  1 57 54 43 39 49  8  0  0 13 50 50 10  0 31 54 43 39 49  6
  1 57 54 43 39 49  8  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0
 37 53 59  1]
[39 56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1
 58 53  1 42 47 43  1 58 46 39 52  1 58 53  1 44 39 51 47 57 46 12  0  0
 13 50 50 10  0 30 43 57 53 50 60 43 42  8  1 56 43 57 53 50 60 43 42  8
  0  0 18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 18 47 56 57 58  6  1
 63 53 59  1]
[56 43  1 39 50 50  1 56 43 57 53 50 60 43 42  1 56 39 58 46 43 56  1 58
 53  1 42

In [8]:
batch_size = 64
buffer_size = 10000

seq_dataset = seq_dataset.shuffle(buffer_size).batch(
    batch_size, drop_remainder=True)


In [9]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.models.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size,None]), 
        keras.layers.LSTM(units=rnn_units, 
                          stateful =True,
                          recurrent_initializer='glorot_uniform', 
                          return_sequences=True), 
        keras.layers.Dense(vocab_size)
    ])
    return model
    
model = build_model(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units,
    batch_size= batch_size
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [10]:
for input_example_batch, target_example_batch in seq_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)

(64, 100, 65)


In [11]:
# random sampling 随机策略
# greedy 贪心策略

sample_indices = tf.random.categorical(logits=example_batch_predictions[0], 
                     num_samples=1)
# print(sample_indices)
sample_indices = tf.squeeze(sample_indices, axis=1)
# print(sample_indices.numpy())
print("Input: ", repr(''.join(idx2char[input_example_batch[0]])) )
print('')
print("Output: ", repr(''.join(idx2char[target_example_batch[0]])) )
print('')
print("Predictions: ", repr(''.join(idx2char[sample_indices])) )

Input:  'noble Paris and true Romeo dead.\nShe wakes; and I entreated her come forth,\nAnd bear this work of he'

Output:  'oble Paris and true Romeo dead.\nShe wakes; and I entreated her come forth,\nAnd bear this work of hea'

Predictions:  "Su!c-e:x?;zBkt'Wj$VKFjBAC.&rAN:ftfNBeYa,yoPRA.BDprYfjhn&EErI J!ZKzI:PCa$.aC;.-ubnjDYo:Yp!I\nmV iG&sOF"


In [12]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True)
 
model.compile(optimizer= 'adam', loss=loss)
example_loss = loss(target_example_batch, example_batch_predictions)

print(example_loss.shape)
print(example_loss.numpy().mean())

(64, 100)
4.1749544


In [13]:
output_dir = './data/text_generation'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
checkpoint_prefix = os.path.join(output_dir, 'ckpt_{epoch}')

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True)

epochs = 50
history = model.fit(seq_dataset, epochs = epochs,
                    callbacks = [checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 9s 53ms/step - loss: 2.5889
Epoch 2/50
172/172 [==============================] - 7s 40ms/step - loss: 1.8782
Epoch 3/50
172/172 [==============================] - 7s 40ms/step - loss: 1.6320
Epoch 4/50
172/172 [==============================] - 7s 40ms/step - loss: 1.5009
Epoch 5/50
172/172 [==============================] - 7s 40ms/step - loss: 1.4215
Epoch 6/50
172/172 [==============================] - 7s 40ms/step - loss: 1.3672
Epoch 7/50
172/172 [==============================] - 7s 40ms/step - loss: 1.3224
Epoch 8/50
172/172 [==============================] - 7s 40ms/step - loss: 1.2835
Epoch 9/50
172/172 [==============================] - 7s 40ms/step - loss: 1.2491
Epoch 10/50
172/172 [==============================] - 7s 40ms/step - loss: 1.2141
Epoch 11/50
172/172 [==============================] - 7s 40ms/step - loss: 1.1800
Epoch 12/50
172/172 [==============================] - 7s 40ms/step - loss: 1.1442
Epoch 13/50
1

In [14]:
tf.train.latest_checkpoint(output_dir)

'./data/text_generation/ckpt_50'

In [15]:
model2 = build_model(vocab_size, embedding_dim, 
                    rnn_units, 
                    batch_size=1)

model2.load_weights(tf.train.latest_checkpoint(output_dir))
model2.build(tf.TensorShape([1, None]))

# start ch sequence A, 
# A -> model -> b
# A.append(b) -> B
# B -> model -> c
# B.append(c) -> C
# C -> model -> d

def generate_text(model, start_string, num_generate=1000):
    input_eval = [char2idx[ch] for ch in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    
    text_generated = []
    model.reset_states()
    
    for _ in range(num_generate):
        #1. model inference -> predictions
        #2. sample -> ch -> text_generated.
        #3. update input_eval
        predictions = model(input_eval)
        
        predictions = tf.squeeze(predictions, 0)
        predicted_id = tf.random.categorical(
        predictions, num_samples= 1)[-1, 0].numpy()
        
        text_generated.append(idx2char[predicted_id])
        input_eval = tf.expand_dims([predicted_id], 0)
        
    return start_string + ''.join(text_generated)
        
new_text = generate_text(model2, "All: ")
print(new_text)

All: yet we will help you to-day?

CHRISTOPHER:
So sadly aftly repent the sin the adverse perchant;
And the new-made be deposed; ven Edward's son;
Thou shalt not live to speak; and happileting to
father:
From my hap monst to do, and so are yet
Be said it may be abhorror!
And, as become of one foul generation, when he lies,
And liberal toe!n if they can behold him.

ROMEO:

SICINIUS:
Let me hear not?
O, my Lord Hastings are the all renowned of an addlcreath?

Nurse:
Marry, well, then.
If I be not, he's head; as it were, in a wretch
That return limit of them, good my lord.

ANGELO:
I would not tit the time; when we saw in this blood of it
his venom these services to want his.

RICH:
The silence oftheir life fairly of my father's life.

JOHN OF GAUNT:
God's!

Third Gentleman:
No: the princess hear no less upon the head,
That covenants subjects now to bear
My sovereign mischief; measure he be agreed;
O, what consperve a gloss terrorant
In soltier ubstrain'd fast: he may come mother.
Hark, 